In [1]:
import pandas as pd

See tables [here][1], copied from "Sample time and total conversion time" (31.4.5.5/680)
in Teensy K20P64M72SF1RM manual.

Using tables, it should be possible to compute total sample and conversion time based on
clock on Bus clock frequency (i.e., `F_BUS`) and selected ADC clock frequency (i.e., `ADACK`).

[1]: ../adc-sample-and-conversion-time.xlsx

In [2]:
df_hsc_adder = pd.read_clipboard()

In [7]:
df_lst_adder = pd.read_clipboard()

In [4]:
df_sfc_adder = pd.read_clipboard()

In [5]:
df_average_number = pd.read_clipboard()

In [6]:
df_bct = pd.read_clipboard()

In [2]:
def generate_adc_configs_frame():
    '''
    Need to copy and paste tables from Excel and run lines above one at a time.
    '''
    df_combined = (df_sfc_adder.set_index('CFG1[ADLSMP]')
                   .join(df_lst_adder.set_index('CFG1[ADLSMP]'), rsuffix='_lst_adder')
                   .reset_index())

    frames = []

    for i, row_i in df_hsc_adder.iterrows():
        frame = df_combined.copy()
        for k, v in row_i.iteritems():
            if k in frame:
                k = k + '_hsc_adder'
            frame[k] = v
        frames.append(frame)

    df_combined = pd.concat(frames)

    frames = []

    for i, row_i in df_average_number.iterrows():
        frame = df_combined.copy()
        for k, v in row_i.iteritems():
            if k in frame:
                k = k + '_average_number'
            frame[k] = v
        frames.append(frame)

    df_combined = pd.concat(frames)

    frames = []

    for i, row_i in df_bct.iterrows():
        frame = df_combined.copy()
        for k, v in row_i.iteritems():
            if k in frame:
                k = k + '_bct'
            frame[k] = v
        frames.append(frame)

    return pd.concat(frames)

In [6]:
import os

if not os.path.isfile('adc_configs.msg'):
    df_adc_configs = generate_adc_configs_frame()
    df_adc_configs.to_msgpack('adc_configs.msg')
#     df_adc_configs.to_hdf('adc_configs.h5', '/adc_configs',
#                           datacolumns=True, complib='zlib', complevel=6)
else:
#     df_adc_configs = pd.read_hdf('adc_configs.h5', '/adc_configs')
    df_adc_configs = pd.read_msgpack('adc_configs.msg')
    F_BUS = 48e6
    ADC_CLK = 22e6

    df_configs_i = df_adc_configs.copy()
    conversion_time = (df_adc_configs['bus clock'] / F_BUS +
                       df_adc_configs.extra_us * 1e-6 +
                       1 / ADC_CLK * (df_adc_configs.ADCK
                                      + df_adc_configs.AverageNum * (df_adc_configs.ADCK_bct + 
                                                                     df_adc_configs.ADCK_lst_adder + 
                                                                     df_adc_configs.ADCK_hsc_adder)))
    df_configs_i['conversion_time'] = conversion_time
    df_configs_i['conversion_rate'] = 1 / conversion_time

    pd.set_eng_float_format(accuracy=2, use_eng_prefix=True)

In [7]:
(df_configs_i.loc[(df_configs_i['Bit-width'] == 13)
                  & (df_configs_i['CFG2[ADHSC]'] == 1)
                  & (df_configs_i['AverageNum'] == 1),
#                   & (df_configs_i['Mode'] == 'single-ended'),  # Filter query
                  ['conversion_time', 'conversion_rate',
                   'Mode', 'Bit-width', 'AverageNum',
                   'CFG2[ADLSTS]',
                  ]  # Fields to look at
                 ].drop_duplicates(subset=['conversion_rate'])
 .sort_values('conversion_rate', ascending=False)
)

,conversion_time,conversion_rate,Mode,Bit-width,AverageNum,CFG2[ADLSTS]
0,1.79u,559.92k,differential,13,1,0
30,1.97u,508.18k,differential,13,1,2
29,2.24u,446.32k,differential,13,1,1
28,2.60u,384.00k,differential,13,1,0
59,6.79u,147.36k,differential,13,1,3
58,6.97u,143.52k,differential,13,1,2
57,7.24u,138.11k,differential,13,1,1
56,7.60u,131.51k,differential,13,1,0


In [5]:
(df_configs_i.loc[(df_configs_i.Mode == 'single-ended') &
                  (df_configs_i.conversion_rate >= 100e3) &
                  (df_configs_i.AverageNum >= 4)]
[['Bit-width', 'AverageNum', 'conversion_rate']]
).drop_duplicates(subset='conversion_rate').sort(['AverageNum', 'conversion_rate'], ascending=False)

C:\Users\Christian\Envs\rpc-dev\lib\site-packages\ipykernel\__main__.py:19: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Bit-width,AverageNum,conversion_rate
0,8,8,167.25k
31,8,8,152.38k
0,8,8,150.47k
0,10,8,143.28k
31,8,8,138.33k
31,10,8,132.23k
0,10,8,130.79k
30,8,8,126.65k
31,10,8,121.52k
30,8,8,116.79k
